# Grouping Companies for non-downtown buildings

In [1]:
from utils import owners

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re
import geopandas as gp
import urllib.parse

In [ ]:
exact_matches_1 = pd.read_csv("exact_matches_1.csv")
exact_matches_1_principals = owners.get_companies_principals(exact_matches_1)

In [ ]:
all_matches = pd.DataFrame([])
for i in range(1, 11):
    print(f"Getting principals for exact_matches_{i}")
    exact_matches = pd.read_csv(f"exact_matches_{i}.csv")
    exact_matches_principals = owners.get_companies_principals(exact_matches)
    all_matches = pd.conact([all_matches, exact_matches_principals])

for i in range(1, 10):
    print(f"Getting principals for potential_matches_{i}")
    potential_matches = pd.read_csv(f"potential_matches_{i}.csv")
    potential_matches = potential_matches[potential_matches['isMatch']==1]
    potential_matches_principals = owners.get_companies_principals(potential_matches)
    all_matches = pd.conact([all_matches, potential_matches_principals])

In [6]:
out_path = os.getcwd()+"comapnies_and_potential_related_ndt.csv"

In [ ]:
owners.group_companies_by_principals(all_matches, out_path)